In [11]:
import argparse
from datetime import datetime
import os
import json
import pandas as pd
import time

import numpy as np
from transformers import GPT2LMHeadModel,AdamW, WarmupLinearSchedule
from torch.utils.tensorboard import SummaryWriter
import torch
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tnrange, tqdm_notebook

from dataset import GPT21024Dataset 
from utils import add_special_tokens, generate_sample, set_seed

In [34]:
filename = "pubmed_gene_go_summary.tsv"
path = os.path.join("pubmed", filename)
pubmed_table = pd.read_csv(path, sep = "\t")
pubmed_table = pubmed_table[pubmed_table['Abstract'].notna()]
pubmed_table = pubmed_table[pubmed_table['Target'].notna()]
pubmed_table.shape

(7652, 5)

In [35]:
pubmed_table.head(5)

,GeneID,PubMed,Target,Title,Abstract
0,12,14668352,SERPINA3 enables DNA binding; located_in nucleus.,The SANT2 domain of the murine tumor cell DnaJ...,The murine tumor cell DnaJ-like protein 1 or M...
1,64215,14668352,DNAJC1 enables ATPase activator activity; invo...,The SANT2 domain of the murine tumor cell DnaJ...,The murine tumor cell DnaJ-like protein 1 or M...
2,12,2404007,SERPINA3 enables serine-type endopeptidase inh...,"Cloning, expression, purification, and biologi...",Human alpha 1-antichymotrypsin has been cloned...
3,13,11481320,AADAC enables lipase activity.,Characterization of the rodent genes for aryla...,"In the current study, we have determined the c..."
4,13,8063807,AADAC enables catalytic activity.,Human liver arylacetamide deacetylase. Molecul...,Microsomal arylacetamide deacetylase (DAC) com...


In [12]:
def write_json(i,article, abstract):
    """ Saves a json file."""

    file = "./pubmed/gpt2_1024_data/"+str(i)+".json"
    js_example = {}
    js_example['id'] = i
    js_example['article'] = article
    js_example['abstract'] = abstract
    with open(file, 'w') as f:
        json.dump(js_example, f, ensure_ascii=False)

In [30]:
print(pubmed_table.iloc[274])

GeneID                                                    183
PubMed                                               17159080
Target      AGT involved_in G protein-coupled receptor sig...
Title       Cross-talk between angiotensin II receptor typ...
Abstract                                                  NaN
Name: 274, dtype: object


In [36]:
tokenizer = add_special_tokens()
train_ids = []
#file_id_map = {}
for index, row in pubmed_table.iterrows():
    article = row["Abstract"]
    summary = row["Target"]
    article = tokenizer.encode(article)
    summary = tokenizer.encode(summary)
    if len(article) > 0 and len(summary) > 0 and (len(article) + len(summary) <= 1023):
        train_ids.append(index)
        write_json(index, article, summary)
        #file_id_map[i] = os.path.basename(file).replace('.story', '')
        if index%100==0:
            print(index, " files written")
            
x,y = int(len(train_ids)*0.8), int(len(train_ids)*0.9)
valid_ids = train_ids[x:y]
test_ids = train_ids[y:]
train_ids = train_ids[:x]
with open("./pubmed/ids.json",'w') as f:
    js = {}
    js['train_ids'] = train_ids
    js['valid_ids'] = valid_ids
    js['test_ids'] = test_ids
    json.dump(js,f)

"""
# file_id_map maps the json file ids to actual cnn/dm file names ending with ".story"
print("saving file_id_map...")

with open("file_id_map.pickle", 'wb') as f:
    pickle.dump(file_id_map,f)
print("file_id_map saved.")
"""

0  files written
100  files written
200  files written
300  files written
400  files written
500  files written
600  files written
700  files written
800  files written
900  files written
1000  files written
1100  files written
1200  files written
1400  files written
1500  files written
1600  files written
1700  files written
1800  files written
1900  files written
2000  files written
2100  files written
2200  files written
2300  files written
2400  files written
2500  files written
2600  files written
2700  files written
2800  files written
2900  files written
3000  files written
3100  files written
3200  files written
3300  files written
3400  files written
3500  files written
3600  files written
3700  files written
3800  files written
3900  files written
4000  files written
4100  files written
4200  files written
4300  files written
4400  files written
4500  files written
4600  files written
4700  files written
4800  files written
4900  files written
5000  files written
5100  files 

'\n# file_id_map maps the json file ids to actual cnn/dm file names ending with ".story"\nprint("saving file_id_map...")\n\nwith open("file_id_map.pickle", \'wb\') as f:\n    pickle.dump(file_id_map,f)\nprint("file_id_map saved.")\n'

In [38]:
#please change default arguments if needed

parser = argparse.ArgumentParser()
parser.add_argument("--lr",default=5e-5, type=float, help="learning rate")
parser.add_argument("--seed",default=42, type=int,  help="seed to replicate results")
parser.add_argument("--n_gpu",default=1, type=int,  help="no of gpu available")
parser.add_argument("--gradient_accumulation_steps",default=2, type=int, help="gradient_accumulation_steps")
parser.add_argument("--batch_size",default=1, type=int,  help="batch_size")
parser.add_argument("--num_workers",default=4, type=int,  help="num of cpus available")
parser.add_argument("--device",default=torch.device('cpu'), help="torch.device object")
parser.add_argument("--num_train_epochs",default=1, type=int,  help="no of epochs of training")
parser.add_argument("--output_dir",default='./output', type=str,  help="path to save evaluation results")
parser.add_argument("--model_dir",default='./weights', type=str,  help="path to save trained model")
parser.add_argument("--max_grad_norm",default=1.0, type=float, help="max gradient norm.")
parser.add_argument("--root_dir",default='./pubmed/gpt2_1024_data', type=str, help="location of json dataset.")
parser.add_argument("--ids_file",default='./pubmed/ids.json', type=str, help="location of train, valid and test file indexes")
args = parser.parse_args([])
print(args)

Namespace(lr=5e-05, seed=42, n_gpu=1, gradient_accumulation_steps=2, batch_size=1, num_workers=4, device=device(type='cpu'), num_train_epochs=1, output_dir='./output', model_dir='./weights', max_grad_norm=1.0, root_dir='./pubmed/gpt2_1024_data', ids_file='./pubmed/ids.json')


In [39]:
def train(args, model, tokenizer, train_dataset, valid_dataset, ignore_index):
    """ Trains GPT2 model and logs necessary details.
        Args:
            args: dict that contains all the necessary information passed by user while training
            model: finetuned gpt/gpt2 model
            tokenizer: GPT/GPT2 tokenizer
            train_dataset: GPT21024Dataset object for training data
            ignore_index: token not considered in loss calculation
    """
    writer = SummaryWriter('./output/logs')
    train_sampler = RandomSampler(train_dataset)
    train_dl = DataLoader(train_dataset,sampler=train_sampler,batch_size=args.batch_size,num_workers=args.num_workers)
    loss_fct = CrossEntropyLoss(ignore_index=ignore_index) #ignores padding token for loss calculation
    optimizer = AdamW(model.parameters(),lr=args.lr)
    scheduler = WarmupLinearSchedule(optimizer,100,80000)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = tnrange(int(args.num_train_epochs), desc="Epoch")
    set_seed(args)
    for _ in train_iterator:
        epoch_iterator = tqdm_notebook(train_dl, desc="Training")
        for step, batch in enumerate(epoch_iterator):
            inputs, labels = batch['article'].to(args.device), batch['article'].to(args.device)
            model.train()
            logits = model(inputs)[0]
            # only consider loss on reference summary just like seq2seq models
            shift_logits = logits[..., batch['sum_idx']:-1, :].contiguous()
            shift_labels = labels[..., batch['sum_idx']+1:].contiguous()
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            loss = loss/args.gradient_accumulation_steps
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1
                writer.add_scalar('lr', scheduler.get_lr()[0], global_step)
                writer.add_scalar('loss', (tr_loss - logging_loss)/args.gradient_accumulation_steps, global_step)
                logging_loss = tr_loss
                print("loss:", loss.item(), end='\n\n')
                if (step + 1)/args.gradient_accumulation_steps == 1.0:
                	print('After 1st update: ', end='\n\n')
                	generate_sample(valid_dataset, tokenizer, model, num=2, eval_step=False,device=args.device)
                
                
            if (step + 1) % (10*args.gradient_accumulation_steps) == 0:
                results = evaluate(args, model, valid_dataset, ignore_index, global_step)
                for key, value in results.items():
                    writer.add_scalar('eval_{}'.format(key), value, global_step)
                print('After', global_step+1,'updates: ', end='\n\n')
                generate_sample(valid_dataset, tokenizer, model, num=2, eval_step=True,device=args.device)


In [40]:
def evaluate(args, model, eval_dataset, ignore_index, global_step=None):
    """ Returns perplexity score on validation dataset.
        Args:
            args: dict that contains all the necessary information passed by user while training
            model: finetuned gpt/gpt2 model
            eval_dataset: GPT21024Dataset object for validation data
            global_step: no. of times gradients have backpropagated
            ignore_index: token not considered in loss calculation
    """
    if not os.path.exists(args.output_dir):
        os.mkdir(args.output_dir)
    eval_output_dir = args.output_dir

    results = {}
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.batch_size)
    loss_fct = CrossEntropyLoss(ignore_index=ignore_index) #ignores padding token for loss calculation

    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
        inputs, labels = batch['article'].to(args.device), batch['article'].to(args.device)
        
        with torch.no_grad():
            logits = model(inputs)[0]
            idx = batch['sum_idx'].item() # index of separator token
            # only consider loss on reference summary just like seq2seq models
            shift_logits = logits[..., batch['sum_idx']:-1, :].contiguous()
            shift_labels = labels[..., batch['sum_idx']+1:].contiguous()
            lm_loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {
        "perplexity": perplexity
    }
    print("perplexity:", perplexity.item())

    if global_step:
        output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
        with open(output_eval_file, "a") as f:
            for key in sorted(result.keys()):
                f.write('\n\n')
                f.write("time = %s, %s = %s, step = %s\n" % (datetime.now().strftime("%d/%m/%Y %H:%M:%S"), key, str(result[key]), str(global_step)))
    return result           

In [41]:
# creating training and validation dataset object

train_data = GPT21024Dataset(args.root_dir,args.ids_file,mode='train',length=3000) #training on only 3000 datasets
valid_data = GPT21024Dataset(args.root_dir,args.ids_file,mode='valid',length=500)  #validation on only 500 datasets

In [42]:
# load pretrained GPT2
tokenizer = add_special_tokens()
ignore_idx = tokenizer.pad_token_id
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))
model.to(args.device)

100%|████████████████████████| 548118077/548118077 [00:34<00:00, 16069161.70B/s]


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [43]:
#training the model

start = time.time()
train(args, model, tokenizer, train_data, valid_data, ignore_idx)
print('total time: ', (time.time()-start)/60, " minutes", end='\n\n')

print('Saving trained model...')
model_file = os.path.join(args.model_dir, 'model_data{}_trained_after_{}_epochs_only_sum_loss_ignr_pad.bin'.format(len(train_data),args.num_train_epochs))
config_file = os.path.join(args.model_dir, 'config_data{}_trained_after_{}_epochs_only_sum_loss_ignr_pad.json'.format(len(train_data),args.num_train_epochs))
torch.save(model.state_dict(), model_file)
model.config.to_json_file(config_file)

/tmp/ipykernel_3148508/2934305216.py:20: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  train_iterator = tnrange(int(args.num_train_epochs), desc="Epoch")


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

/tmp/ipykernel_3148508/2934305216.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  epoch_iterator = tqdm_notebook(train_dl, desc="Training")


Training:   0%|          | 0/3000 [00:00<?, ?it/s]

/home/rli10/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1050.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


...

  0%|          | 0/100 [00:00<?, ?it/s]

new_article

The cellular effects of stromal cell-derived factor-1 (SDF-1) are mediated primarily by binding to the CXC chemokine receptor-4. We report in this study that SDF-1 and its peptide analogues induce a concentration- and time-dependent accumulation of phosphatidylinositol-(3,4,5)-trisphosphate (PtdIns(3,4,5)P3) in Jurkat cells. This SDF-1-stimulated generation of D-3 phosphoinositide lipids was inhibited by pretreatment of the cells with an SDF-1 peptide antagonist or an anti-CXCR4 Ab. In addition, the phosphoinositide 3 (PI 3)-kinase inhibitors wortmannin and LY294002, as well as the Gi protein inhibitor pertussis toxin, also inhibited the SDF-1-stimulated accumulation of PtdIns(3,4,5)P3. The effects of SDF-1 on D-3 phosphoinositide lipid accumulation correlated well with activation of the known PI 3-kinase effector protein kinase B, which was also inhibited by wortmannin and pertussis toxin. Concentrations of PI 3-kinase inhibitors, sufficient to inhibit PtdIns(3,4,5)P3 acc

Evaluating:   0%|          | 0/500 [00:00<?, ?it/s]

perplexity: 2.906761187957438e+30
After 11 updates: 



TypeError: generate_sample() missing 1 required positional argument: 'model'